In [ ]:
# ============================================
# Inference / Validation (Cleaned)
#  - Neutral defaults & small utils
#  - Budget-banded candidate retrieval
#  - Feature computation (same as training, no masking)
#  - Feature alignment to model columns
#  - Optional location-sensitive re-ranking
#  - Recommend & pretty-print
# ============================================

from __future__ import annotations
import numpy as np
import pandas as pd
import json
import json
from joblib import load as joblib_load
from recommendation_utils import (
    budget_affinity_score, compute_match_scores_gaussian,
    compute_env_priority_scores, load_pa_similarity_matrix,
    pa_location_similarity_with_sensitivity
)

In [ ]:
# --------------------------------------------------------
# 0) Small utilities
# --------------------------------------------------------
def _neutral_defaults() -> dict:
    """Neutral fallbacks when a feature is missing at inference."""
    return {"sim_budget": 0.8, "sim_location": 0.6, "default": 0.5}

def _pick(row, *candidates, default=""):
    """Pick the first present column from candidates."""
    for c in candidates:
        if c in row and pd.notna(row[c]):
            return row[c]
    return default

# --------------------------------------------------------
# 1) Candidate retrieval (budget band ±20%)
# --------------------------------------------------------
def build_inference_candidates(items_df: pd.DataFrame,
                               budget: float,
                               k_candidates: int = 300,
                               seed: int = 2025) -> pd.DataFrame:
    """
    Retrieve k candidates within ±20% of budget; fallback to global sample.
    """
    band = items_df[(items_df["resale_price"] >= 0.80 * budget) &
                    (items_df["resale_price"] <= 1.20 * budget)]
    if len(band) < k_candidates:
        return items_df.sample(min(k_candidates, len(items_df)), random_state=seed)
    return band.sample(k_candidates, random_state=seed)

# --------------------------------------------------------
# 2) Feature computation (same sims as training; no masking)
# --------------------------------------------------------
def compute_features_for_user_items(user_row: pd.Series | dict,
                                    items_df: pd.DataFrame,
                                    sim_df_pa: pd.DataFrame,
                                    item_place_col: str = "Plan") -> pd.DataFrame:
    """
    Build the sim_* features on each (user, item) and keep rich item attributes
    for explanation. No masking at inference.
    """
    user = pd.Series(user_row) if not isinstance(user_row, pd.Series) else user_row
    rows = []
    neutr = _neutral_defaults()

    for _, it in items_df.iterrows():
        # --- sims used by model ---
        feats = {}
        feats["sim_budget"] = budget_affinity_score(user.get("Budget_SGD"), it["resale_price"])
        feats |= compute_match_scores_gaussian(user, it)     # sim_area, sim_floor, sim_newhome
        feats |= compute_env_priority_scores(user, it)       # sim_park_access, sim_bus_access, sim_mrt_access, sim_amenities, sim_school
        feats["sim_location"] = pa_location_similarity_with_sensitivity(
            user_place=user.get("Preferred_Place", None),
            item_place=it.get(item_place_col, None),
            sim_df=sim_df_pa,
            distance_sensitivity=user.get("Priority_Distance_Proximity", None),
            default_when_missing=0.0,
            neutral=neutr["sim_location"]
        )

        # --- carry item attributes for printing ---
        block  = _pick(it, "block")
        street = _pick(it, "street_name", "street_nam")
        town   = _pick(it, "town")
        pa     = _pick(it, item_place_col)
        storey = _pick(it, "storey_range")
        area   = _pick(it, "floor_area_sqm")
        lease  = _pick(it, "remaining_lease", "remaining_lease_mths", "remaining_lease_months")

        # nearby facility counts / flags
        def _int(x): 
            return int(x) if pd.notna(x) else 0
        rows.append({
            "item_id": it.get("item_id", it.get("id", _)),
            "block": block, "street": street, "town": town, "pa": pa,
            "storey_range": storey,
            "floor_area_sqm": area, "remaining_lease": lease,
            "resale_price": it.get("resale_price", np.nan),
            "mrt_200": _int(it.get("mrt_200", np.nan)),
            "mrt_500": _int(it.get("mrt_500", np.nan)),
            "bus_200": _int(it.get("bus_200", np.nan)),
            "bus_500": _int(it.get("bus_500", np.nan)),
            "MALL_500M": _int(it.get("MALL_500M", np.nan)),
            "HWKR_500M": _int(it.get("HWKR_500M", np.nan)),
            "HOSP_1K": _int(it.get("HOSP_1K", np.nan)),
            "GP_SCH_1K": _int(it.get("GP_SCH_1K", np.nan)),
            "GP_SCH_2K": _int(it.get("GP_SCH_2K", np.nan)),
            "PK_500M_IN": _int(it.get("PK_500M_IN", np.nan)),
            # model features (no masking at inference)
            **feats,
            "sim_budget_missing": 0,
            "sim_location_missing": 0,
            "sim_mrt_access_missing": 0,
            "sim_bus_access_missing": 0,
            "sim_amenities_missing": 0,
            "sim_school_missing": 0,
            "sim_area_missing": 0,
            "sim_floor_missing": 0,
            "sim_newhome_missing": 0,
        })

    return pd.DataFrame(rows)

# --------------------------------------------------------
# 3) Ensure feature alignment to model columns
# --------------------------------------------------------
def ensure_feature_alignment(df_feats: pd.DataFrame, feature_cols: list[str]) -> pd.DataFrame:
    """
    Guarantee df contains every model feature; fill sims with neutrals and flags with 0.
    """
    neutr = _neutral_defaults()
    for c in feature_cols:
        if c not in df_feats.columns:
            if c.endswith("_missing"):
                df_feats[c] = 0
            elif c == "sim_budget":
                df_feats[c] = neutr["sim_budget"]
            elif c == "sim_location":
                df_feats[c] = neutr["sim_location"]
            else:
                df_feats[c] = neutr["default"]
    return df_feats[feature_cols]

# --------------------------------------------------------
# 4) Optional: location-sensitive re-ranking (inference-time)
# --------------------------------------------------------
def rerank_with_location_sensitivity(df_scored: pd.DataFrame,
                                     user_distance_sensitivity: float,
                                     loc_col: str = "sim_location",
                                     pred_col: str = "score",
                                     out_col: str = "score_final",
                                     hard_pref_place: str | None = None,
                                     boost_threshold: float = 0.75,
                                     boost_value: float = 0.05):
    """
    Blend model score with location similarity:
        score_final = (1 - alpha) * score + alpha * sim_location
    where alpha grows with user sensitivity. Optionally boost near-preferred places.
    """
    alpha = 0.3 + 0.5 * float(np.clip(user_distance_sensitivity or 0.0, 0.0, 1.0))
    df = df_scored.copy()
    df[out_col] = (1.0 - alpha) * df[pred_col] + alpha * df[loc_col]

    if hard_pref_place is not None:
        mask = df[loc_col] >= float(boost_threshold)
        df.loc[mask, out_col] += float(boost_value)
    return df.sort_values(out_col, ascending=False), alpha

# ---------- Utility helpers ----------
def _pick_metric(row: pd.Series, candidates: list[str]) -> float:
    """Try several possible column names; return the first valid float value."""
    for c in candidates:
        if c in row and pd.notna(row[c]):
            try:
                return float(row[c])
            except Exception:
                pass
    return np.nan

def _should_block_top1(row: pd.Series,
                       sim_threshold: float = 0.30,
                       model_threshold: float = 1.0,
                       na_counts_as_fail: bool = True) -> tuple[bool, dict]:
    """Return (should_block, value_dict)."""
    budget = _pick_metric(row, ["sim_budget", "budget", "budget_sim"])
    loc    = _pick_metric(row, ["sim_location", "location", "loc"])
    area   = _pick_metric(row, ["sim_area", "area", "area_sim"])
    model  = _pick_metric(row, ["score", "Model", "model", "pred", "predict_score"])

    vals = {"budget": budget, "loc": loc, "area": area, "model": model}

    if na_counts_as_fail and (np.isnan(model) or any(np.isnan(v) for v in [budget, loc, area])):
        return True, vals

    bad_sim   = any(v < sim_threshold for v in [budget, loc, area] if not np.isnan(v))
    bad_model = (not np.isnan(model)) and (model < model_threshold)
    return (bad_sim or bad_model), vals

# ---------- Helper functions ----------
def _pick_metric(row: pd.Series, candidates: list[str]) -> float:
    """Try multiple candidate column names and return the first valid float value."""
    for c in candidates:
        if c in row and pd.notna(row[c]):
            try:
                return float(row[c])
            except Exception:
                pass
    return np.nan

def _should_block_top1(row: pd.Series,
                       sim_threshold: float = 0.30,
                       model_threshold: float = 1.0,
                       na_counts_as_fail: bool = True) -> tuple[bool, dict]:
    """Check whether Top-1 should be blocked based on similarity and model thresholds."""
    budget = _pick_metric(row, ["sim_budget", "budget", "budget_sim"])
    loc    = _pick_metric(row, ["sim_location", "location", "loc"])
    area   = _pick_metric(row, ["sim_area", "area", "area_sim"])
    model  = _pick_metric(row, ["score", "Model", "model", "pred", "predict_score"])
    vals = {"budget": budget, "loc": loc, "area": area, "model": model}

    # If NaNs exist, optionally count as failure
    if na_counts_as_fail and (np.isnan(model) or any(np.isnan(v) for v in [budget, loc, area])):
        return True, vals

    bad_sim   = any(v < sim_threshold for v in [budget, loc, area] if not np.isnan(v))
    bad_model = (not np.isnan(model)) and (model < model_threshold)
    return (bad_sim or bad_model), vals

# ---------- Main function ----------
def recommend_for_user(model,
                       feature_cols: list[str],
                       user_input: dict,
                       items_df: pd.DataFrame,
                       sim_df_pa: pd.DataFrame,
                       item_place_col: str = "Plan",
                       topn: int = 10,
                       k_candidates: int = 300,
                       seed: int = 2025,
                       use_location_rerank: bool = True,
                       sim_threshold=0.4,
                       model_threshold=1.0,
                       warn_threshold: float = 0.30) -> pd.DataFrame:
    """
    Full inference pipeline:
      1. Retrieve candidate properties
      2. Compute user-item features
      3. Predict ranking scores
      4. (Optional) location-sensitive re-ranking
      5. Produce a structured JSON-like payload for LLMs (and print summary)
    """
    # 1) Candidate retrieval
    budget = float(user_input["Budget_SGD"])
    cand = build_inference_candidates(items_df, budget, k_candidates=k_candidates, seed=seed)

    # 2) Feature computation
    feats_df = compute_features_for_user_items(user_input, cand, sim_df_pa, item_place_col=item_place_col)

    # 3) Model prediction
    X = ensure_feature_alignment(feats_df.copy(), feature_cols)
    feats_df["score"] = model.predict(X)

    # 4) Optional location re-ranking
    if use_location_rerank:
        user_p = float(user_input.get("Priority_Distance_Proximity", 0.0))
        pref_place = user_input.get("Preferred_Place", None)
        feats_df, alpha_used = rerank_with_location_sensitivity(
            feats_df, user_distance_sensitivity=user_p,
            loc_col="sim_location", pred_col="score", out_col="score_final",
            hard_pref_place=pref_place, boost_threshold=0.75, boost_value=0.05
        )
        rank_col = "score_final"
    else:
        alpha_used = None
        rank_col = "score"

    recs = feats_df.sort_values(rank_col, ascending=False).head(topn).copy()

    # ---------- Construct base structured payload ----------
    user_summary = {
        k: user_input.get(k, None) for k in [
            "Budget_SGD","Preferred_Flat_Area","Preferred_Place","Floor_Preference","NewHome_Preference",
            "Priority_MRT_Access","Priority_Bus_Access","Priority_Amenities",
            "Priority_School_Proximity","Priority_Park_Access","Priority_Distance_Proximity"
        ]
    }
    result_payload = {
        "status": None,
        "meta": {
            "topn": int(topn),
            "use_location_rerank": bool(use_location_rerank),
            "location_alpha": float(alpha_used) if alpha_used is not None else None,
            "thresholds": {
                "block_sim_threshold": float(sim_threshold),
                "block_model_threshold": float(model_threshold),
                "warn_threshold": float(warn_threshold),
            },
            "user": user_summary,
        },
        "fail_reasons": None,   # Explanation if blocked or no result
        "items": []        # Structured property results
    }

    # ---------- Step 1: No candidates ----------
    if recs.empty:
        result_payload["status"] = "no_result"
        result_payload["fail_reasons"] = {
            "type": "no_candidates_after_rerank",
            "message": "No suitable properties found under current preferences. Try increasing budget or relaxing filters."
        }
        print(json.dumps(result_payload, ensure_ascii=False, indent=2))
        return recs

    # ---------- Step 2: Top-1 fails quality threshold ----------
    need_block, vals = _should_block_top1(recs.iloc[0],
                                          sim_threshold=sim_threshold,
                                          model_threshold=model_threshold)
    if need_block:
        result_payload["status"] = "blocked_top1"
        result_payload["reasons"] = {
            "type": "top1_fails_quality_threshold",
            "triggered_values": {k: (None if np.isnan(v) else float(v)) for k, v in vals.items()},
            "required": {"sim_min": float(sim_threshold), "model_min": float(model_threshold)},
            "message": "Top-1 fails quality threshold; consider increasing budget, expanding preferred location, or loosening size/type preference."
        }
        print(json.dumps(result_payload, ensure_ascii=False, indent=2))
        return recs.head(0)

    # ---------- Step 3: Build structured items ----------
    def _safe_get(obj, name, default=None):
        try:
            v = getattr(obj, name)
            return v if v is not None else default
        except Exception:
            return default

    sim_fields = [
        "sim_budget","sim_location","sim_area",
        "sim_mrt_access","sim_bus_access","sim_amenities","sim_school","sim_floor","sim_newhome"
    ]

    # Small helper to pack nearby-facilities info
    nearby_pack = lambda r: {
        "mrt_200": int(_safe_get(r, "mrt_200", 0) or 0),
        "mrt_500": int(_safe_get(r, "mrt_500", 0) or 0),
        "bus_200": int(_safe_get(r, "bus_200", 0) or 0),
        "bus_500": int(_safe_get(r, "bus_500", 0) or 0),
        "hawker_500m": int(_safe_get(r, "HWKR_500M", 0) or 0),
        "mall_500m": int(_safe_get(r, "MALL_500M", 0) or 0),
        "school_1km": int(_safe_get(r, "GP_SCH_1K", 0) or 0),
        "school_2km": int(_safe_get(r, "GP_SCH_2K", 0) or 0),
        "park_500m_in": int(_safe_get(r, "PK_500M_IN", 0) or 0),
    }

    items_struct = []
    for i, r in enumerate(recs.itertuples(index=False), 1):
        addr = " ".join(str(x) for x in [_safe_get(r, "block", ""), _safe_get(r, "street", "")] if str(x))
        loc  = _safe_get(r, "pa", "") or _safe_get(r, "town", "")

        # Collect similarity scores
        sims = {}
        for s in sim_fields:
            if s in recs.columns:
                val = getattr(r, s, np.nan)
                sims[s.replace("sim_", "")] = None if pd.isna(val) else float(val)

        base_score  = _safe_get(r, "score", np.nan)
        final_score = _safe_get(r, rank_col, np.nan)

        # Identify features below warn threshold
        warn_keys = ["mrt_access", "bus_access", "amenities", "school", "floor"]
        low_flags = [k for k in warn_keys if (k in sims and sims[k] is not None and sims[k] < float(warn_threshold))]

        item_entry = {
            "rank": i,
            "attributes": {
                "address": addr,
                "location": loc,
                "flat_type": _safe_get(r, "flat_type", ""),
                "storey_range": _safe_get(r, "storey_range", ""),
                "floor_area_sqm": _safe_get(r, "floor_area_sqm", None),
                "remaining_lease": _safe_get(r, "remaining_lease", None),
                "resale_price": None if pd.isna(_safe_get(r, "resale_price", np.nan)) else float(_safe_get(r, "resale_price", np.nan)),
                "nearby": nearby_pack(r),
            },
            "scores": {
                "model": None if pd.isna(base_score) else float(base_score),
                "final": None if pd.isna(final_score) else float(final_score),
                "loc": None if pd.isna(_safe_get(r, "sim_location", np.nan)) else float(_safe_get(r, "sim_location", np.nan)),
                "sims": sims,
                "low_flags": low_flags,  # which features fall below warn_threshold
            }
        }
        items_struct.append(item_entry)

    result_payload["status"] = "ok"
    result_payload["items"] = items_struct

    # ---------- Print structured payload and human-readable preview ----------
    print(json.dumps(result_payload, ensure_ascii=False, indent=2))

    print("\n=== Human-readable preview ===")
    print(f"User: budget={user_summary['Budget_SGD']}, place={user_summary['Preferred_Place']}, "
          f"area={user_summary['Preferred_Flat_Area']}, floor_pref={user_summary['Floor_Preference']}, "
          f"newhome_pref={user_summary['NewHome_Preference']}")
    if use_location_rerank:
        print(f"location alpha: {alpha_used:.2f}")
    print(f"TOP-{topn} recommendations")

    for item in items_struct:
        a, s = item["attributes"], item["scores"]
        print(f"\n#{item['rank']} | {a['address']} [{a['location']}]")
        print(f"   Type/Floor: {a['flat_type']} / {a['storey_range']}")
        if a["floor_area_sqm"] or a["remaining_lease"]:
            print(f"   Area: {a['floor_area_sqm']} sqm   Lease: {a['remaining_lease']}")
        print(f"   Price: ${0 if a['resale_price'] is None else a['resale_price']:,.0f} | "
              f"Model: {s['model']:.4f} | Loc: {0 if s['loc'] is None else s['loc']:.2f} | Final: {s['final']:.4f}")

        eval_keys = ["mrt_access","bus_access","amenities","school","floor"]
        eval_pairs = []
        for k in eval_keys:
            val = s["sims"].get(k)
            if val is None:
                eval_pairs.append(f"{k}: NA")
            else:
                eval_pairs.append(f"{k}: {val:.2f}")

        print("   match eval → " + " | ".join(eval_pairs))
        if s["low_flags"]:
            print(f"   ⚠ unmet features (sim<{warn_threshold:.2f}): {', '.join(s['low_flags'])}")

    # Return DataFrame (you can change to 'return result_payload' if you want to return structured data directly)
    return recs

In [ ]:
# =========================
# Inference main
# =========================
def main_infer(
    # Fallback paths if artifacts is None:
    model_path: str | None = None,              # e.g. "ranker_lgbm.joblib"
    feature_cols: str | None = None,
    item_place_col: str = "PLAN",
    topn: int = 10,
    k_candidates: int = 300,
    seed: int = 2025,
    use_location_rerank: bool = True,
    items_path: str = None,
    pa_sim_path: str = None,
    user_input: dict | None = None,
):
    """
    Run a full inference pass with a partially specified user profile.
    - If `artifacts` is provided (from training), it uses it directly.
    - Otherwise it loads model and feature_cols from disk.
    """
    # 1) Load model + feature_cols
    model = joblib_load(model_path)
            
        

    # 2) Load item matrix & PA similarity
    items_df = pd.read_csv(items_path)
    sim_df_pa = load_pa_similarity_matrix(pa_sim_path)

    # 3) Mock a partially specified user (some fields intentionally omitted)
    #    - Missing fields will be handled by neutral defaults inside feature builders.
    if user_input is None:
        user_input = {
            "Budget_SGD": 1100000,                 # known & reliable
            "Preferred_Flat_Area": 100,              # wants 90 sqr
            "Preferred_Place": "JURONG EAST",      # explicit place
            "Priority_Distance_Proximity": 0.8,  # very distance-sensitive
            # The following are partially specified / masked on purpose:
            "Floor_Preference": 2,                # provided
            "NewHome_Preference": 0.5,            # neutral on new vs resale
            # priorities: leave some missing to test neutral handling
            "Priority_MRT_Access": 5,
            "Priority_Bus_Access": 5,
            "Priority_Amenities": 5,
            "Priority_School_Proximity": 5,
            "Priority_Park_Access": 5
        }

    # 4) Run recommendation (this calls: retrieval → features → predict → rerank → print)
    recs = recommend_for_user(
        model=model,
        feature_cols=feature_cols,
        user_input=user_input,
        items_df=items_df,
        sim_df_pa=sim_df_pa,
        item_place_col=item_place_col,
        topn=topn,
        k_candidates=k_candidates,
        seed=seed,
        use_location_rerank=True,
    )

    
    #fi = pd.DataFrame({"feature": feature_cols, "gain": model.feature_importances_}).sort_values("gain", ascending=False)
    
    imp_gain  = model.booster_.feature_importance(importance_type="gain")
    imp_split = model.booster_.feature_importance(importance_type="split")
    names     = model.booster_.feature_name()

    importance_df = pd.DataFrame({
    "feature": names,
    "gain": imp_gain,
    "split": imp_split
    }).sort_values("gain", ascending=False)

#     print("\n============================")
#     print("Feature Importance (sorted by gain)")
#     print("============================")
#     print(importance_df.to_string(index=False))

    return {"user_input": user_input, "recs": recs, "feature_cols": feature_cols, "model": model}


# ---------------------------
# Script entry (example)
# ---------------------------
if __name__ == "__main__":
    feature_cols=['sim_budget', 'sim_budget_missing', 'sim_location', 'sim_location_missing', 'sim_mrt_access', 'sim_mrt_access_missing', 'sim_bus_access', 'sim_bus_access_missing', 'sim_amenities', 'sim_amenities_missing', 'sim_school', 'sim_school_missing', 'sim_area', 'sim_area_missing', 'sim_floor', 'sim_floor_missing', 'sim_newhome', 'sim_newhome_missing', 'sim_park_access', 'sim_park_access_missing']
    deploy_path = r"./ranker_lgbm.joblib"
    result = main_infer(
        model_path=deploy_path,
        feature_cols=feature_cols,
        items_path=r"../../../data/item_matrix_merged.csv",
        pa_sim_path=r"../../../data/PA_centroid_similarity_0_1.csv", # Call the distance matrix table
        item_place_col="Plan",
        topn=10,
        k_candidates=300,
        use_location_rerank=True
    )

{
  "status": "ok",
  "meta": {
    "topn": 10,
    "use_location_rerank": true,
    "location_alpha": 0.7,
    "thresholds": {
      "block_sim_threshold": 0.4,
      "block_model_threshold": 1.0,
      "warn_threshold": 0.3
    },
    "user": {
      "Budget_SGD": 1100000,
      "Preferred_Flat_Area": 100,
      "Preferred_Place": "JURONG EAST",
      "Floor_Preference": 2,
      "NewHome_Preference": 0.5,
      "Priority_MRT_Access": 5,
      "Priority_Bus_Access": 5,
      "Priority_Amenities": 5,
      "Priority_School_Proximity": 5,
      "Priority_Park_Access": 5,
      "Priority_Distance_Proximity": 0.8
    }
  },
  "fail_reasons": null,
  "items": [
    {
      "rank": 1,
      "attributes": {
        "address": "459 CLEMENTI AVE 3",
        "location": "CLEMENTI",
        "flat_type": "",
        "storey_range": "16 TO 18",
        "floor_area_sqm": 110.0,
        "remaining_lease": "77 years 01 month",
        "resale_price": 1020000.0,
        "nearby": {
          "mrt_200

In [ ]:
from recommendation_utils import main_infer

user_input = {
    "Budget_SGD": 1100000,                 # known & reliable
    "Preferred_Flat_Area": 100,              # wants 90 sqr
    "Preferred_Place": "JURONG EAST",      # explicit place
    "Priority_Distance_Proximity": 0.8,  # very distance-sensitive
    # The following are partially specified / masked on purpose:
    "Floor_Preference": 2,                # provided
    "NewHome_Preference": 0.5,            # neutral on new vs resale
    # priorities: leave some missing to test neutral handling
    "Priority_MRT_Access": 5,
    "Priority_Bus_Access": 5,
    "Priority_Amenities": 5,
    "Priority_School_Proximity": 5,
    "Priority_Park_Access": 5
}

if __name__ == "__main__":
    feature_cols = ['sim_budget', 'sim_budget_missing', 'sim_location', 'sim_location_missing', 'sim_mrt_access', 'sim_mrt_access_missing', 'sim_bus_access', 'sim_bus_access_missing', 'sim_amenities', 'sim_amenities_missing', 'sim_school', 'sim_school_missing', 'sim_area', 'sim_area_missing', 'sim_floor', 'sim_floor_missing', 'sim_newhome', 'sim_newhome_missing', 'sim_park_access', 'sim_park_access_missing']
    deploy_path = r"./ranker_lgbm.joblib"
    result = main_infer(
        model_path=deploy_path,
        feature_cols=feature_cols,
        items_path=r"../../../data/item_matrix_merged.csv",
        pa_sim_path=r"../../../data/PA_centroid_similarity_0_1.csv", # Call the distance matrix table
        item_place_col="Plan",
        topn=10,
        k_candidates=300,
        use_location_rerank=True,
        user_input=user_input
    )

{
  "status": "ok",
  "meta": {
    "topn": 10,
    "use_location_rerank": true,
    "location_alpha": 0.7,
    "thresholds": {
      "block_sim_threshold": 0.4,
      "block_model_threshold": 1.0,
      "warn_threshold": 0.3
    },
    "user": {
      "Budget_SGD": 1100000,
      "Preferred_Flat_Area": 100,
      "Preferred_Place": "JURONG EAST",
      "Floor_Preference": 2,
      "NewHome_Preference": 0.5,
      "Priority_MRT_Access": 5,
      "Priority_Bus_Access": 5,
      "Priority_Amenities": 5,
      "Priority_School_Proximity": 5,
      "Priority_Park_Access": 5,
      "Priority_Distance_Proximity": 0.8
    }
  },
  "fail_reasons": null,
  "items": [
    {
      "rank": 1,
      "attributes": {
        "address": "459 CLEMENTI AVE 3",
        "location": "CLEMENTI",
        "flat_type": "",
        "storey_range": "16 TO 18",
        "floor_area_sqm": 110.0,
        "remaining_lease": "77 years 01 month",
        "resale_price": 1020000.0,
        "nearby": {
          "mrt_200